In [1]:
from keras.models import load_model
import cv2
import numpy as np
from random import choice
import time
import os, random
import glob
#from PIL import Image
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from tkinter import *
from tkinter.ttk import Progressbar
from tkinter import ttk

Using TensorFlow backend.


# Game Interface

In [7]:
def startGameInterface():
    ###### Options ########
    gameTitle = "Welcome to Follow My Lead"
    #backgroundImagePath = "photobg3.gif"
    backgroundImageColor = "black"
    windowAddedHeight = 55
    welcomeText = "Welcome To Follow My Lead"; welcomeTextBgColor="black";  welcomeTextColor = "white"
    buttonText = "Start Game"; buttonBgColor = "black"; buttonTextColor="white";
    progressBarWaitSeconds = 2;

    ###### Create Window ######
    window = Tk()  # Tk 클래스 객체(window) 생성
    window.resizable(False, False)
    window.title(gameTitle)

    ###### Assign Background Image #####
    #photoBg = PhotoImage(file=backgroundImagePath)
    photoBg = PhotoImage(file="a.png")
    window.geometry(str(photoBg.width())+'x'+str(photoBg.height()+windowAddedHeight))
    window.configure(background=backgroundImageColor)
    Label(window, image=photoBg).grid(row=0, column=0, sticky=W)  # , bg="black"

    ###### Assign Welcome Text #####
    Label(window, text=welcomeText, bg=welcomeTextBgColor, fg=welcomeTextColor, font="none 12 bold").grid(row=1, column=0, sticky=W)

    ###### Button ############
    def clicked():
        oldTime = time.time(); finishTime = oldTime + progressBarWaitSeconds
        while (time.time() < finishTime):
            bar['value'] = 100 - int(100 *(finishTime-time.time())/progressBarWaitSeconds)
            window.update_idletasks()
        window.destroy()
        global openGame; openGame=1
    btn = Button(window, text=buttonText,  bg=buttonBgColor, fg=buttonTextColor, font=("Arial Bold", 10), command=clicked)  #, padx=5, pady=5
    btn.grid(column=0, row=0)

    ###### Progress Bar ######
    style = ttk.Style()
    style.theme_use('default')
    style.configure("black.Horizontal.TProgressbar", background='black')
    bar = Progressbar(window, length=400, style='black.Horizontal.TProgressbar')
    bar['value'] = 0
    bar.grid(column=0, row=2)

    #### Start Loop ######
    window.mainloop()

In [8]:
def makeSureBackGroundClearConfirm():

    ######## Options #########
    fontColor = (255, 255, 0); fontScale = 1;  lineType = 2; font = cv2.FONT_HERSHEY_SIMPLEX
    textPositionOriginal = (10, 100)
    welcomeText = 'Ready and Press Enterkey !!!!!!'
    betweenLineSpace = 100

    welcomeTextArray = welcomeText.split('/n')
    
    videoCam = cv2.VideoCapture(0) # index : 카메라 장치 번호 (0 부터 차례로 증가)
    
    # Check web cam is working
    if videoCam.isOpened():       # 캡쳐 객체 연결 확인
        # ret : 프레임 읽기 성공 또는 실패 여부, True / False
        # originalVid : 프레임 이미지, Numpy 배열 또는 None
        ret, originalVid = videoCam.read()     # 다음 프레임 읽기
    else:
        ret = False
        
    # Original Video
    while ret:  # 프레임 읽기가 성공 했다면~
        ret, originalVid = videoCam.read()
        #### Write Text ####
        textPosition = textPositionOriginal
        for i in range(len(welcomeTextArray)):
            cv2.putText(originalVid, welcomeTextArray[i], textPosition, font, fontScale, fontColor, lineType)
            textPositionList = list(textPosition)     #Had to do this to change element in tuple
            textPositionList[1] += betweenLineSpace;
            textPosition = tuple(textPositionList)
        cv2.imshow("Original", originalVid)
        inputKey = cv2.waitKey(1)  # 1ms 지연(40fps로 가정) 
        if inputKey == 13:
            global openGame; openGame=2
            break;
        elif inputKey == 27:
            break;
    cv2.destroyAllWindows()
    videoCam.release()

# Main

In [9]:
def timeOnScreen(originalVid, t ,score):
    fontColor = (255, 255, 255);
    fontScale = 1;
    lineType = 2;
    font = cv2.FONT_HERSHEY_SIMPLEX

    timeText = "Time Left " + str(int(t))
    scoreText="Score "+str(score)
    timePosition = (60, 100)
    scorePosition= (60,150)
    cv2.putText(originalVid, timeText, timePosition, font, fontScale, fontColor, lineType)
    cv2.putText(originalVid, scoreText, scorePosition, font, fontScale, fontColor, lineType)

In [10]:
def showimage1(random_number, t, score):
    icon = cv2.imread('random_img/{}.jpg'.format(random_number))
    icon = cv2.resize(icon, (400, 380))
    
    timeOnScreen(icon, t ,score)
    
    cv2.imshow('Pose', icon)

In [11]:
def main():
    model = load_model('model.h5')
    img_dim = 128
    class_labels = ['1', '2', '3']
    
    cap = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX
    t0=time.time()
    gameType=0
    random_number = choice(['1', '2', '3'])
    score=0
    while True:
        ret, frame = cap.read()
    
        # webcam space
        cv2.rectangle(frame, (100, 100), (500, 500), (255, 255, 255), 2)
        roi = frame[0:500, 0:500]
        img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_dim, img_dim))
    
        ## 포즈 예측##
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = img.astype('float32')/255
        pred = np.argmax(model.predict(img)) 
        color = (0,0,255)
        cv2.putText(frame, class_labels[pred], (50,50), font, 1.0, color, 2) 
        
    
        currentTime = time.time()
        timeDiff = currentTime - t0
        displayTime = 5
        #score=0
        t=0
        gameType ==0
        
        
        cv2.imshow('Webcam', np.fliplr(frame))
        #print(class_labels[pred])
        if class_labels[pred] == random_number:
            if timeDiff > displayTime: 
                score += 1
            
                #print("matching")
            
        showimage1(random_number, displayTime-timeDiff, score)
        if timeDiff > displayTime:
            #cv2.destroyWindow('IMG')
            t0 = time.time()
            random_number = choice(['1', '2', '3'])
            #print('game over') 
        
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    print("score :", score)
    cap.release()
    cv2.destroyAllWindows()  

In [12]:
openGame = 0;   # If you wanna skip interface, assign this to 2

startGameInterface()

if(openGame==1):
    makeSureBackGroundClearConfirm()

if(openGame==2):
    if __name__=="__main__":
        main()

score : 2
